In [1]:
# Command Prompt installs
#
# For BeautifulSoup
# ...
#
# For ChromeDriverManager
# pip install webdriver-manager

from bs4 import BeautifulSoup
import requests
import csv
import time

import os
import pickle
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # Useful for sending keys (not necessarily used below) #cyh
from selenium.webdriver.common.action_chains import ActionChains # Useful for queuing commands (not necessarily used below) #cyh
from webdriver_manager.chrome import ChromeDriverManager # requires pip install webdriver-manager

In [2]:
# This will scrape pages from Semantics Scholar
# I filter down to journal articles only. I also put the search term into quotation marks.

# "International Affairs": 
# https://www.semanticscholar.org/search?publicationType%5B0%5D=JournalArticle&q=%22international%20affairs%22&sort=relevance

In [3]:
# Old version using CSV (now uses pkl files)
#with open("Paper_data1.csv","w",newline= "") as csvfile:
#    writer= csv.writer(csvfile)
#    writer.writerow(["Source", "Abstract","Keywords", "Journal", "Authors", "Date", "General"])
#    #rows =zip(source, abstract_list, keyword_list, author_list, general_list)

paper_df = pd.DataFrame(index=range(10000), columns=['Title', 'Abstract', 'Keyword', 'Journal', 'Author', 'Date', 'General', 'Source'])


In [4]:
# Change the URL here!
# I filter down to journal articles only. I also put the search term into quotation marks.

url= "https://www.semanticscholar.org/search?publicationType%5B0%5D=JournalArticle&q=%22international%20affairs%22&sort=relevance"

In [5]:
# Easier to open each entry and scrape from it's full page
# Need to script Selenium for that
#
# To open papers click: <a href=...   data-selenium-selector="title-link">
#
# To expand full abstract need to click <a class="text-truncator__toggle mod-clickable more" role="button" 
#                                       aria-label="Expand truncated text" data-selenium-selector="text-truncator-toggle">
#                                       CONTINUE READING</a>
#
# Then do that for each paper down the page.
# 10 per page
# Then can manually change pages (or automate it if can figure out how)
# End of url is &page=2, sequential for each new page
# Grab first 20 pages, 10 entries per page, per topic?

In [6]:
#Test to see if it works
#url= "https://www.semanticscholar.org/paper/The-Logic-of-New-Media-in-International-Affairs-Kluver/a8c68376fe0011ddfd5b10c413de6fe7264fd485"

#response= requests.get(url)
#time.sleep(10)
#soup = BeautifulSoup(response.text, "lxml")
#abstract= soup.find_all(class_= "text-truncator abstract__text text--preline")
#keywords= soup.find_all(class_ ="entity-columns")
#author= soup.find_all(class_="author-list__link author-list__author-name")
#general= soup.find_all(class_="fresh-paper-detail-page__header")


#date= soup.find_all(data-selenium-selector="paper-year")
#date= soup.find("paper-year")
#date= soup.select(data-selenium-selector.paper-year)

In [7]:
# WEBSCRAPING CELL
#
# From each page:
# Title will be in:<h1 data-selenium-selector="paper-detail-title"></h1>
# class= fresh-paper-detail-page__header
# Title, author, journal, year, volume, pages
# Abstract will be in:<div class="text-truncator abstract__text text--preline">
# Keywords will be in: <div class="entity-columns" data-selenium-selector="entities-list"><div class="flex-container flex-wrap"><div class="flex-item"><ul class="entity-column">
#                      <li class="entity-group-item" data-selenium-selector="entity-name" data-heap-id="entity_click"><span class="preview-box__target">
#                       <div class="flex-item"><ul class="entity-column">
# Journal will be in: <span data-selenium-selector="venue-metadata"><span><span>New Media &amp; Society</span></span></span>
# Authors names will be in: <a class="author-list__link author-list__author-name"
# Date will be in: <span data-selenium-selector="paper-year">

# Better version if your computer is able to detect the chromedriver in the install folder; mine cannot #cyh
#chrome_driver = "/Applications/chromedriver"
#os.environ["webdriver.chrome.driver"] = chrome_driver
#driver = webdriver.Chrome(chrome_driver)
driver = webdriver.Chrome(ChromeDriverManager().install())

#source_list = []
#abstract_list =[]
#keyword_list =[]
#author_list = []
#general_list= []

# This is the beginning URL of all papers on Semantics Scholar
url_base = "https://www.semanticscholar.org/paper/"

paper_counter = 0
paper_counter_temp = 0



# Loop through X Results Pages
max_page_results = 2
for i in range(1,max_page_results+1): # add 1 due to starting at index 1
    url_page= url + "&page=" + str(i)
    driver.get(url_page)
    print("i = " + str(i) + " " + driver.current_url)
    time.sleep(2)    
    
    # Get the paper titles
    search_results = driver.find_elements_by_class_name("search-result-title") 
    paper_counter_temp = paper_counter
    for search_result_counter in range(0, len(search_results)):
        paper_df['Title'][paper_counter_temp] = search_results[search_result_counter].text
        paper_counter_temp = paper_counter_temp + 1
    
    
    hyperlinks = driver.find_elements_by_tag_name("a")
    
    # Get the URLs to the papers
    paper_url_list = []
    for hyperlink_counter in range(0, len(hyperlinks)):
        hyperlink_url = hyperlinks[hyperlink_counter].get_attribute("href")
        # Check if this element had a URL
        if hyperlink_url is not None:
            # Paper urls begin with "https://www.semanticscholar.org/paper/" as defined in url_base
            if hyperlink_url[0:len(url_base)] == url_base:
                paper_url_list.append(hyperlink_url)
            
    
    print("Number of papers are " + str(len(paper_url_list)))
    # try next sibling as an option
    # better:  try find_all("a") and look for the href
    
    # Loop through each search result on the search results page
    for j in range(0, len(paper_url_list)):
    #for j in range(0, 2):
        # Repeated definition of search_results avoids stale element error so it is purposely repeated
        #search_results = driver.find_elements_by_class_name("search-result-title") 
        #print("Number of results are " + str(len(search_results)))
        # try
        
        # Try to click on the next Title.  Sometimes the title is blocked by the banner, so scroll down to make it visible
        #try:
        #    search_results[j].click()
        #except:
        #    #actions = ActionChains(driver)
        #    #actions.move_to_element(search_results[j]).perform()
        #    #time.sleep(1)
        #    # Scroll down to the Title of interest
        #    driver.execute_script("arguments[0].scrollIntoView();", search_results[j])
        #    # Scroll back up slightly as the upper part of the site covers what we scrolled to
        #    driver.execute_script("window.scrollBy(0, -150)", "")
        #    #time.sleep(1)
        #    search_results[j].click()
        
        paper_url_list
        # "Click" on the link by going directly to the URL. Directly going avoids problems associated with clicking, such as
        # banners/etc covering the hyperlinks and inconsistent website behavior.
        
        driver.get(paper_url_list[j])
        print("j = " + str(j) + " while 'clicking' on jth search result " + driver.current_url)
        response = requests.get(driver.current_url)
        time.sleep(1)
        
        ## Locate the "Continue Reading" button to expand the abstract and click it.  If no such button found, then do nothing.
        # This functionality is likely unnecessary if the correct Abstract source is found since the abstract in full is
        # available in another webelement.  
        #try:
        #    clickable_more_button = driver.find_element_by_partial_link_text("CONTINUE READING")
        #    clickable_more_button.click()
        #except:
        #    pass
        #time.sleep(1)
        
        soup = BeautifulSoup(response.text, "lxml")
        
        # Need to get the title
        
        # #cyh The abstracts location is incorrect as it only pulls the truncated abstract regardless.  Need to go back later
        # and find it properly.  See <meta name="description" underneath the <title>.  If my guess is correct, it probably
        # wasn't even necessary to click on the "CONTINUE READING" button to get the whole abstract.  This means we may be
        # able to remove that "click on CONTINUE READING" code and save ourselves some wait time
        abstracts= soup.find_all(class_= "text-truncator abstract__text text--preline")
        keywords= soup.find_all(class_ ="entity-columns")
        authors= soup.find_all(class_="author-list__link author-list__author-name")
        generals= soup.find_all(class_="fresh-paper-detail-page__header")
    
        
        #source_list.append(driver.current_url)
        #for abstract in abstracts:
        #    abstract_list.append(abstract.text)
        #for keyword in keywords:
        #    keyword_list.append(keyword.text)
        #for author in authors:
        #    author_list.append(author.text)
        #for general in generals:
        #    general_list.append(general.text) 
        
        # Paper Titles are added near the top of the Search Results page loop
        if len(abstracts) > 0:
            paper_df['Abstract'][paper_counter] = abstracts[0].text
        if len(keywords) > 0:
            paper_df['Keyword'][paper_counter] = keywords[0].text
        if len(authors) > 0:
            paper_df['Author'][paper_counter] = authors[0].text
        if len(generals) > 0:
            paper_df['General'][paper_counter] = generals[0].text
        paper_df['Source'][paper_counter] = driver.current_url
        paper_counter = paper_counter + 1


        #source= [url_page]*len(abstract_list)
        
        # Return to Search Results page
        driver.back()
        if driver.current_url != url_page:
            driver.get(url_page)
        
        time.sleep(1)
        print("After returning to Results Page (equivalent of clicking 'back'), url is " + driver.current_url)
        print("end of j iteration " + str(j))
    
        
    
    # Purposely write the results as they come rather than all at once
    #with open("Paper_data1.csv", "a", newline= "") as csvfile:
    #    wr =csv.writer(csvfile)
    #    rows =zip(source_list, abstract_list, keyword_list, author_list, general_list)
    #    wr.writerows(rows)
        
    print("end of i iteration " + str(i))

    
# Purposely write the results as they come rather than all at once
paper_df.to_pickle("./paper_data.pkl")

# Preview part of what was saved.
paper_df.head(20)

print("done")
# Use NLPK, Spacey, etc later for nlp
# put into dataframe
# or changing the encoding of the csv


Checking for win32 chromedriver:76.0.3809.126 in cache
Driver found in C:\Users\548253\.wdm\chromedriver\76.0.3809.126\win32/chromedriver.exe
i = 1 https://www.semanticscholar.org/search?publicationType%5B0%5D=JournalArticle&q=%22international%20affairs%22&sort=relevance&page=1
Number of papers are 10
j = 0 while 'clicking' on jth search result https://www.semanticscholar.org/paper/International-Affairs-Portal%3A-A-Semantic-Web-Contreras-Benjamins/d4177744d80786b8c0a742b11e59a054811f5bd1
After returning to Results Page (equivalent of clicking 'back'), url is https://www.semanticscholar.org/search?publicationType%5B0%5D=JournalArticle&q=%22international%20affairs%22&sort=relevance&page=1
end of j iteration 0
j = 1 while 'clicking' on jth search result https://www.semanticscholar.org/paper/The-Logic-of-New-Media-in-International-Affairs-Kluver/a8c68376fe0011ddfd5b10c413de6fe7264fd485
After returning to Results Page (equivalent of clicking 'back'), url is https://www.semanticscholar.org/

After returning to Results Page (equivalent of clicking 'back'), url is https://www.semanticscholar.org/search?publicationType%5B0%5D=JournalArticle&q=%22international%20affairs%22&sort=relevance&page=2
end of j iteration 8
j = 9 while 'clicking' on jth search result https://www.semanticscholar.org/paper/National-narcissism%3A-Internal-dimensions-and-Cai-Gries/bde3412f77cea9446313831131690b6a683c15a9
After returning to Results Page (equivalent of clicking 'back'), url is https://www.semanticscholar.org/search?publicationType%5B0%5D=JournalArticle&q=%22international%20affairs%22&sort=relevance&page=2
end of j iteration 9
end of i iteration 2
done


In [10]:
# DATA PROCESSING CELL
# Continue coding here.

# Read in data (especially if the webscraping cell execution was skipped)
paper_df = pd.read_pickle("./paper_data.pkl")


In [147]:
# This cell for testing access to web elements

url_page= url + "&page=" + "2"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url_page)
time.sleep(2)
#soupResultsPage = BeautifulSoup(driver.page_source, "html.parser")
search_results = driver.find_elements_by_class_name("search-result-title")
#search_results[1].click()
#time.sleep(1)

#driver.back()
#time.sleep(1)
#search_results = driver.find_elements_by_class_name("search-result-title")
#search_results[2].click()



Checking for win32 chromedriver:76.0.3809.126 in cache
Driver found in C:\Users\548253\.wdm\chromedriver\76.0.3809.126\win32/chromedriver.exe


In [ ]:
#EVERYTHING BELOW THIS CELL IS SAMPLE CODE.  It's not for use in our NLP code except as an example to imitate.

In [12]:
url_page= url 
# for a later step

In [9]:
response= requests.get(url_page)

In [10]:
time.sleep(1)

In [11]:
soup = BeautifulSoup(response.text, "lxml")

In [12]:
quotes=soup.find_all(class_="text")
authors= soup.find_all(class_= "author")

In [13]:
# this just tests if it works.
#print(quotes)

In [14]:
# This also tests if it works.
# print(authors)

In [15]:
print(len(quotes))

0


In [16]:
quote_list =[]
author_list=[]

In [ ]:
for quote in quotes:
    quote_list.append(quote.text)

In [ ]:
for author in authors:
    author_list.append(author.text)

In [ ]:
source= [url_page]*len(quote_list)

In [ ]:
# with open("quote_info.csv", "a", newline= "") as csvfile:
#     wr =csv.writer(csvfile)
#     rows =zip(source, quote_list, author_list)
#     wr.writerows(rows)

In [ ]:
# This also tests if it works.
# quote_list